In [5]:
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
!pip install datasets
from datasets import load_dataset
from nltk.tokenize import ToktokTokenizer
from sklearn.metrics import f1_score
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Deep Average Network для определения сентимента 

В этой домашке мы будет классифицировать твиты на 3 тональности.  
Вы будете использовать предобученные эмбеддинги слов, так что для начала обязательно нужно посмотреть [туториал по их использованию](https://github.com/BobaZooba/DeepNLP/blob/master/Tutorials/Word%20vectors%20%26%20Data%20Loading.ipynb).

Наши классы:  

Индекс | Sentiment  
-- | --  
0 | negative  
1 | neutral  
2 | positive  

Вам предстоит реализовать такую модель:
![Архитектура модели DAN](https://www.researchgate.net/profile/Shervin-Minaee/publication/340523298/figure/fig1/AS:878252264550411@1586403065555/The-architecture-of-the-Deep-Average-Network-DAN-10.ppm)

Что она из себя представляет:
- Мы подаем в нее индексы слов
- Переводим индексы слов в эмбеддинги
- Усредняем эмбеддинги
- Пропускаем усредненные эмбеддинги через `Multilayer Perceptron`

В этой домашке вам предстоит:
- Перевести тексты в матрицы с индексами токенов
- Реализовать модель
- Обучить ее
- Понять хорошо ли вы это сделали

Это очень важная модель, потому что она очень простая и показывает достаточно высокие метрики. В дальнейшем на работе советую использовать такую модель как бейзлайн. И в качестве эмбеддингов слов взять эмбеддинги от берта/роберты/тд.

## 🤗 Datasets
В этом туториале мы будем использовать подготовленные данные из библиотеки [datasets](https://github.com/huggingface/datasets). Мы вряд ли еще будем пользоваться этой библиотекой, так как нам будет важно самим подготавливать данные. Во-первых, для простоты, во-вторых, здесь есть достаточно неплохие практики. [Здесь](https://huggingface.co/datasets) вы сможете найти достаточно большое количество различных датасетов. Возможно, когда-нибудь они вам пригодятся.

## Загрузите эмбеддинги слов
Реализуйте функцию по загрузке эмбеддингов из файла. Она должна отдавать словарь слов и `np.array`
Формат словаря:
```python
{
    'aabra': 0,
    ...,
    'mom': 6546,
    ...
    'xyz': 100355
}
```
Формат матрицы эмбеддингов:
```python
array([[0.44442278, 0.28644582, 0.04357426, ..., 0.9425766 , 0.02024289,
        0.88456545],
       [0.77599317, 0.35188237, 0.54801261, ..., 0.91134102, 0.88599103,
        0.88068835],
       [0.68071886, 0.29352313, 0.95952505, ..., 0.19127958, 0.97723054,
        0.36294011],
       ...,
       [0.03589378, 0.85429694, 0.33437761, ..., 0.39784873, 0.80368014,
        0.76368042],
       [0.01498725, 0.78155695, 0.80372969, ..., 0.82051826, 0.42314861,
        0.18655465],
       [0.69263802, 0.82090775, 0.27150426, ..., 0.86582747, 0.40896573,
        0.33423976]])
```

Количество строк в матрице эмбеддингов должно совпадать с размером словаря, то есть для каждого токена должен быть свой эмбеддинг. По параметру `num_tokens` должно брать не более указано в этом параметре количество токенов в словарь и матрицу эмбеддингов.

## Загружаем данные из библиотеки
Мы сразу получим `torch.utils.data.Dataset`, который сможем передать в `torch.utils.data.DataLoader`

In [6]:
dataset_path = "tweet_eval"
dataset_name = "sentiment"

train_dataset = load_dataset(path=dataset_path, name=dataset_name, split="train")
valid_dataset = load_dataset(path=dataset_path, name=dataset_name, split="validation")
test_dataset = load_dataset(path=dataset_path, name=dataset_name, split="test")

## `torch.utils.data.DataLoader`

In [7]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

## Посмотрим что отдает нам `Loader`
Это батч формата:
```python
batch = {
    "text": [
        "text1",
        "text2",
        ...,
        "textn"
    ],
    "label": tensor([
        1,
        1,
        ...,
        0
    ])
}
```
То есть у нас есть словарь с двумя ключами `text` и `label`, где хранится n примеров. То есть для 5-го примера в батче текст будет храниться в `batch["text"][5]`, а индекс класса будет храниться в `batch["label"][5]`.

In [8]:
for batch in train_loader:
    break

batch

{'text': ['"when the workers at Dunkin say ""see you tomorrow"" @user @user',
  'Captain Dubs! David Wright gives the Mets back the lead with an RBI single to make it 6-5 Mets in the 7th inning'],
 'label': tensor([1, 2])}

## Collate
Сейчас перед нами стоит проблема: мы получаем тексты в виде строк, а нам нужны тензоры (матрицы) с индексами токенов, к тому же нам нужно западить последовательности токенов, чтобы все сложить в торчовую матрицу. Мы можем сделать это двумя способами:
- Достать из `train/valid/test_dataset` данные и написать свой `Dataset`, где внутри будет токенизировать текст, токены будут переводиться в индексы и затем последовательность будет падиться до нужной длины
- Сделать функцию, которая бы дополнительно обрабатывали наши батчи. Она вставляется в `DataLoader(collate_fn=<ВАША_ФУНКЦИЯ>)`

## Если вы хотите сделать свой `Dataset`
То вы можете достать данные таким образом.

In [9]:
len(train_dataset["text"]), len(train_dataset["label"])

(45615, 45615)

In [10]:
train_dataset["text"][:2]

['"QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"',
 '"Ben Smith / Smith (concussion) remains out of the lineup Thursday, Curtis #NHL #SJ"']

In [11]:
train_dataset["label"][:2]

[2, 1]

## Если вы хотите сделать `collate_fn`

### Давайте посмотрим что вообще происходит внутри этого метода
Для этого сделаем функцию `empty_collate`, которая принимает на вход батч и отдает его, ничего с ним не делая

In [12]:
def empty_collate(batch):
    return batch

In [13]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=empty_collate)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=empty_collate)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=empty_collate)

In [14]:
for batch in train_loader:
    break

batch

[{'text': '"Is Carly Fiorina Better Off Being Left Out?: Despite her rise in the polls, the GOP contender still may not ma...',
  'label': 1},
 {'text': "Mayweather ends feud with Rousey: 'I wish her nothing but the best'",
  'label': 1},
 {'text': '"Stylistically this is a horrible match up for Rousey, look at the stats. I won\'t be surprised if Rousey takes the L in January #UFC195"',
  'label': 0},
 {'text': "ICYMI: CNN's criteria for the next Republican debate may end up leaving Carly Fiorina out.",
  'label': 0}]

## Формат батча
```python
batch = [
    {
        "text": "text1",
        "label": 0
    }, 
    {
        "text": "text2",
        "label": 1
    },
    ...,
    {
        "text": "textn",
        "label": 1
    }
]
```
То есть теперь у нас есть список, где каждый элемент — это словарь со значениями `text` и `label`.  

Вы можете сделать функцию или класс с методом `collate`. Этот способ решения домашки предодчтительней, так как использовать `collate` очень хорошая практика.

Что я предлагаю:
- Сделайте класс `Tokenizer`

In [15]:
class Tokenizer:
    
    def __init__(self, base_tokenizer, token2index, pad_token, unk_token, max_length):

        print("Tokenizer initializing...")
        self._base_tokenizer = base_tokenizer  # например ToktokTokenizer()

        self.token2index = token2index  # словарь из load_embeddings()
        
        print("Setting up the padding token...")
        self.pad_token = pad_token
        if not self.pad_token in token2index.keys():
            token2index[self.pad_token] = len(token2index)
        self.pad_index = self.token2index[self.pad_token]
        
        print("Setting up the unknown token...")
        self.unk_token = unk_token
        if not self.unk_token in token2index.keys():
            token2index[self.unk_token] = len(token2index)
        self.unk_index = self.token2index[self.unk_token]
        
        self.max_length = max_length

        print("Initialization finished")

    def tokenize(self, text):
        """
        В этом методе нужно разделить строку текста на токены
        """
        return self._base_tokenizer.tokenize(text) # returns text.split()*
            
    
    def indexing(self, tokenized_text):
        """
        В этом методе нужно перевести список токенов в список с индексами этих токенов
        """
        index_list = []
        for token in tokenized_text:
                index_list.append(self.token2index[token] if token in token2index.keys() \
                    else self.token2index["<unk>"])
        return index_list
        

    def padding(self, tokens_indices):
        """
        В этом методе нужно сделать длину tokens_indices равной self.max_length
        Опционально убрать повторяющиеся unk'и
        """
        while len(tokens_indices) != self.max_length:
            tokens_indices.append(self.token2index[self.pad_token])
        
    
    def __call__(self, text):
        """
        В этом методе нужно перевести строку с текстом в вектор с индексами слов нужно размера (self.max_length)
        """
        tokens = self.tokenize(text)
        index_vector = []
        for token in tokens:
            if token in token2index.keys():
                index_vector.append(token2index[token])
        
        self.padding(index_vector)

        return index_vector
        
    def collate(self, batch):
        """
        batch sample {
            text: str
            label: int
        }
        """
        tokenized_texts = list()
        labels = list()
        
        for sample in batch:
            labels.append(sample['label'])
            tokens = self.tokenize(sample['text'])
            indices = self.indexing(tokens)
            self.padding(indices)
            tokenized_texts.append(indices)

            
        tokenized_texts = torch.Tensor(tokenized_texts)  # перевод в torch.Tensor
        labels = torch.Tensor(labels)  # перевод в torch.Tensor
        
        return tokenized_texts, labels

In [16]:
def load_embeddings(path, num_tokens=100_000):
    """
    {label: int, text: str}
    """

    # token2index: Dict[str, int] = {
    #     "<pad>": 0,
    #     "<unk>": 1
    # }
    token2index: Dict[str, int] = {}
    embeddings_matrix: np.array = []

    with open(path, 'r') as file:
        vocab_size, emb_dim = file.readline().strip().split()
        emb_dim = int(emb_dim)
        vocab_size = int(vocab_size)

        num_tokens = min(num_tokens, vocab_size)

        progress_bar = tqdm(total=num_tokens, disable=False, desc='Reading embeddings file')

        for line in file:
            content = line.strip().split()

            token = ' '.join(content[:-emb_dim]).lower()

            if token in token2index:
                continue

            word_vector = np.array(list(map(float, content[-emb_dim:])))
            token2index[token] = len(token2index)
            embeddings_matrix.append(word_vector)

            progress_bar.update()

            if len(token2index) == num_tokens:
                break

        progress_bar.close()

        embeddings_matrix = np.stack(embeddings_matrix)
    
    assert(len(token2index) == embeddings_matrix.shape[0])
    
    return token2index, embeddings_matrix

In [17]:
tokenizer = Tokenizer(ToktokTokenizer(), load_embeddings(path="cc.ru.300.vec")[0], '<pad>', '<unk>', 100)
tokenizer

Reading embeddings file: 100%|██████████| 100000/100000 [00:15<00:00, 6301.88it/s]


Tokenizer initializing...
Setting up the padding token...
Setting up the unknown token...
Initialization finished


## Перед реализацией выбранного метода
Советую, чтобы в итоге `Loader` отдавал кортеж с двумя тензорами:
- `torch.Tensor` с индексами токенов, размерность `(batch_size, sequence_length)`
- `torch.Tensor` с индексами таргетов, размерность `(batch_size)`

То есть, чтобы было так:
```python
for x, y in train_loader:
    ...

>> x
>> tensor([[   37,  3889,   470,  ...,     0,     0,     0],
           [ 1509,   581,   144,  ...,     0,     0,     0],
           [ 1804,   893,  2457,  ...,     0,     0,     0],
           ...,
           [  170, 39526,  2102,  ...,     0,     0,     0],
           [ 1217,   172, 28440,  ...,     0,     0,     0],
           [   37,    56,   603,  ...,     0,     0,     0]])

>> y
>> tensor([1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 2, 0, 0, 1,
           0, 2, 1, 1, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 0, 1, 0,
           1, 0, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 2, 1, 0, 0, 2, 2,
           2, 1, 2, 0, 2, 2, 0, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 0, 2, 2,
           2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 2, 1, 2, 1,
           2, 1, 1, 2, 2, 1, 1, 2])

>> x.shape
>> torch.Size([128, 64])

>> y.shape
>> torch.Size([128])
```
При условии, что батч сайз равен 128, а максимальная длина последовательности равна 64.

In [18]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
!gzip -d cc.ru.300.vec.gz

--2022-12-02 12:05:25--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  42.4MB/s    in 30s     

2022-12-02 12:05:56 (41.7 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]

gzip: cc.ru.300.vec already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [23]:
token2index, embeddings = load_embeddings(path="cc.ru.300.vec")
tokenizer = Tokenizer(ToktokTokenizer(), token2index, '<pad>', '<unk>', 300)

Reading embeddings file: 100%|██████████| 100000/100000 [00:09<00:00, 10243.86it/s]


Tokenizer initializing...
Setting up the padding token...
Setting up the unknown token...
Initialization finished


In [24]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=tokenizer.collate)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=tokenizer.collate)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=tokenizer.collate)

In [25]:
train_loader

In [26]:
for x, y in train_loader:
    break

print(x, y)

tensor([[9.0000e+00, 1.0000e+05, 4.0290e+04, 2.7900e+02, 3.4200e+03, 1.6332e+04,
         3.4884e+04, 0.0000e+00, 1.0000e+05, 8.2780e+03, 6.6500e+02, 4.4892e+04,
         2.1720e+03, 9.1720e+03, 2.7900e+02, 4.5560e+03, 1.0000e+05, 1.0000e+05,
         1.0000e+05, 4.9622e+04, 6.6500e+02, 1.1581e+04, 7.8370e+03, 7.2300e+02,
         1.0000e+05, 1.0000e+05, 1.0000e+05, 6.6500e+02, 3.5000e+01, 1.0000e+05,
         1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05,
         1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05,
         1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05,
         1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05,
         1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05,
         1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05,
         1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05, 1.0000e+05,
         1.0000e+05, 1.0000e

In [27]:
assert(isinstance(x, torch.Tensor))
assert(len(x.size()) == 2)

assert(isinstance(y, torch.Tensor))
assert(len(y.size()) == 1)

# Реализация DAN

На вход модели будут подавать индексы слов

Шаги:
- Переводим индексы слов в эмбеддинги
- Усредняем эмбеддинги
- Пропускаем усредненные эмбеддинги через `Multilayer Perceptron`
    - Нужно реализовать самому

### Что нужно сделать в домашке с точки зрения архитектуры сети:
- Реализовать skip-connection (residual connection) в линейном слое
- Написать свой слой, в котором будут (порядок слоев ниже напутан, так что сами подумайте в каком порядке стоит расположить эти слои) :
  - `Dropout`
  - `BatchNorm` / `LayerNorm`
  - `Residual`, если вы не меняете размерность векторов
  - Функция активации
  - Линейный слой

### Опциональные задания:
- Использовать токенизатор и слой эмбеддингов от предобученного трансформера из библиотеки `transformers`
- Сделать усреднение эмбеддингов с учетом падов
  - Мы используем пады, чтобы сделать единую длину последовательностей в батче
    - То есть у нас максимальная длина в батче, например, 16 токенов, поэтому ко всем последовательностям, у которых длина ниже мы добавляем `16 - len(sequence)` падов
  - То есть получается так, что усредненный вектор предложения зависит от максимальный длины в батче, потому что
    - Среднее вектора `[1, 2, 3]` будет `2`. Среднее вектора `[1, 2, 3, 0, 0]` будет `1.2`
    - Получается, что усредняя с падами мы получаем "неправильный" вектор
  - То есть наши предсказания будут зависеть от того сколько падов у нас есть в предложении
  - Когда мы будем использовать нашу сетку в реальном процессе, скорее всего, мы будем подавать в нее по одному примеру, где падов не будет
    - То есть получается мы будем использовать нашу модель не в той же среде, как и обучали
      - Потому что наши входы меняются, мы не используем пады, результат усреднения другой
    - Это называется `distribution shift`, то есть когда мы учимся на одних данных, а используем на других
      - Это не всегда плохо, потому что иногда только так мы и можем учиться, например, когда мало данных нужного домена
      - Это плохо тогда, когда мы вносим это "случайно", например, как с неправильным усреднением, то есть это своебразный баг


## До обучения
- Выберите метрику(ки) качества и расскажите почему она(они)
    - Обычно есть основная метрика, по которой принимаем решения какие веса брать и дополнительные, которые нам помогут делать выводы, например, о том все ли хорошо с нашими данными, хорошо ли модель справляется с дисбалансом классов и тд
- Эту домашку можно сделать и на `CPU`, но на `GPU` будет сильно быстрее
    - Во всех остальных домашках мы будем учить модели на `GPU`
    - Рано или поздно вам придется посмотреть этот [туториал](https://www.youtube.com/watch?v=pgk1zGv5lU4)
    - Вы можете обучаться на `colab`, это бесплатно

## До эпохи
- Сделайте списки/словари/другое, чтобы сохранять нужные данные для расчета метрик(и) по всей эпохе для трейна и валидации

## Во время эпохи
- Используйте [`tqdm`](https://github.com/tqdm/tqdm) как прогресс бар, чтобы понимать как проходит ваше обучение
- Логируйте лосс
- Логируйте метрику(ки) по батчу
- Сохраняйте то, что вам нужно, чтобы посчитать метрик(и) на всю эпоху для трейна и валидации

## После эпохи
- Посчитайте метрик(и) на всю эпоху для трейна и валидации

## После обучения
- Провалидируйтесь на тестовом наборе и посмотрите метрики
- Постройте [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)
- Постройте графики:
    - [Confusion Matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix)
    - [Опционально] Распределение вероятностей мажоритарного класса (то есть для какого-то примера мы выбираем такой класс и вероятность этого выбора такая-то) на трейне/тесте/валидации
        - Если класс был выбран верно и если была ошибка
- Подумайте что еще вам будет полезно для того, чтобы ответить на такие вопросы: 
    - Что в моделе можно улучшить?
    - Все ли хорошо с моими данными?
    - Все ли хорошо с валидацией?
    - Не переобучился ли я?
    - Достаточно ли я посмотрел на данные?
    - Нужно ли мне улучшить предобработку данных?
    - Нужно ли поменять токенизацию или эмбеддинги?
    - Нет ли у меня багов в реализации?
    - Какие типичные ошибки у моей модели?
    - Как я могу их исправить?

In [28]:
token2index, embeddings = load_embeddings("cc.ru.300.vec")

Reading embeddings file: 100%|██████████| 100000/100000 [00:09<00:00, 10378.96it/s]


In [29]:
class DeepAverageNetwork(nn.Module):
    # out_dim = 3, потому что 3 класса: негативный, нейтральный и позитивный
    def __init__(self, emb_dim, embeddings = torch.Tensor(embeddings), out_dim = 3, token2index = token2index) -> None:
        super().__init__()
        self.token2index = token2index
        self.emb_dim = emb_dim
        self.out_dim = out_dim
        self.vocab_size = len(token2index)
        self.embeddings = nn.Embedding(self.vocab_size, self.emb_dim).from_pretrained(embeddings)
        
        self.linear1 = torch.nn.Linear(self.emb_dim, self.emb_dim)
        self.linear2 = torch.nn.Linear(self.emb_dim, self.emb_dim)
        self.relu = torch.nn.ReLU()
        self.norm = torch.nn.BatchNorm1d(self.emb_dim)
        self.dropout = torch.nn.Dropout()
        self.activate = torch.nn.Softmax(1)

    def forward(self, x):
        norm = self.norm(x)
        x = self.relu(self.linear1(norm))
        x = self.dropout(x)
        x = self.relu(self.linear2(x)) + norm
        x = self.activate(x)
        return x

In [30]:
model = DeepAverageNetwork(300)

In [31]:
model

DeepAverageNetwork(
  (embeddings): Embedding(100000, 300)
  (linear1): Linear(in_features=300, out_features=300, bias=True)
  (linear2): Linear(in_features=300, out_features=300, bias=True)
  (relu): ReLU()
  (norm): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (activate): Softmax(dim=1)
)

## Функция потерь и оптимизатор

In [32]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Сделайте цикл обучения

In [34]:
NUM_EPOCHS = 5  # Задайте количество эпох
# Добавить .to(device) чтобы учить на гпу?
best_epoch = -1
best_test_loss = 1e5
f1_score = -1e5
for n_epoch in range(NUM_EPOCHS):
    # TRAIN
    print(f"Running epoch {n_epoch}, best test loss: {best_test_loss} on epoch {best_epoch}")
    step = 0
    tr_loss = 0
    model.train()

    pbar = tqdm(train_loader)
    for i, (x, y) in enumerate(pbar):
        if len(y) < 2:
            continue
        step += 1
        optimizer.zero_grad()
        y_pred = model(x)
        y_hat = torch.Tensor(y_pred.detach().numpy().argmax(1).tolist())
        y_hat.requires_grad_()
        loss = criterion(y, y_hat)
        loss.backward()
        tr_loss += loss.item()
        optimizer.step()
    tr_loss /= step
    
    # TEST
    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        pbar = tqdm(test_loader)
        for i, (x, y) in enumerate(pbar):
            if len(y) < 2:
                continue
            step += 1
            y_pred = model(x)
            y_hat = torch.Tensor(y_pred.detach().numpy().argmax(1).tolist())
            y_hat.requires_grad_()
            loss = criterion(y, y_hat)
            te_loss += loss.item()
        te_loss /= step

    if te_loss < best_test_loss:
        best_test_loss = te_loss
        best_ep = n_epoch
        # torch.save(model.state_dict(), f"best_model.pt")
    print(f"epoch {n_epoch}, tr_loss {tr_loss}, te_loss {te_loss}")


    
# VALIDATE
print("Validating...")
step = 0
val_loss = 0
with torch.no_grad():
    model.eval()
    for i, (x, y) in enumerate(valid_loader):
        if len(y) < 2:
            continue
        step += 1
        y_pred = model(x)
        y_hat = torch.Tensor(y_pred.detach().numpy().argmax(1).tolist())
        y_hat.requires_grad_()
        loss = criterion(y, y_hat)
        val_loss += loss.item()
    val_loss /= step
print(f"val_loss = {val_loss}")
        



Running epoch 0, best test loss: 100000.0 on epoch -1


100%|██████████| 6142/6142 [00:06<00:00, 888.74it/s]


epoch 0, tr_loss 25.103817397305594, te_loss 254.56480364487405
Running epoch 1, best test loss: 254.56480364487405 on epoch -1


100%|██████████| 6142/6142 [00:05<00:00, 1024.81it/s]


epoch 1, tr_loss 25.148217200697808, te_loss 221.69678385395136
Running epoch 2, best test loss: 221.69678385395136 on epoch -1


...

epoch 2, tr_loss 25.20002547205971, te_loss 87.43389377575362
Running epoch 3, best test loss: 87.43389377575362 on epoch -1


100%|██████████| 6142/6142 [00:06<00:00, 1023.02it/s]


epoch 3, tr_loss 25.12268932206641, te_loss 85.26144983061204
Running epoch 4, best test loss: 85.26144983061204 on epoch -1


100%|██████████| 6142/6142 [00:06<00:00, 1018.90it/s]


epoch 4, tr_loss 25.242419897533207, te_loss 79.21180605849548
Validating...
val_loss = 80.4844709854126
